# LSTM Train - Trafik - PCA

In [ ]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split

In [ ]:
DATASET_PATH = config.DATA_TRAFIK
df = pd.read_parquet(DATASET_PATH / 'data.parquet')
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [ ]:
PCA_COMPONENTS = 5
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=10, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=10)
predictor = TrafficPredictor('lstm', dataset.sensors, X_STEP, Y_STEP, pca_model=dataset.pca if True else None)
predictor.train(train_loader, val_loader, epochs=10, save_path=config.PROJECT_ROOT.joinpath('models/lstm_trafik_pca.pt'), save_optimizer=True)
predictor.show()